In [ ]:
!pip install fitz
!pip install PyMuPDF
!pip install tabula-py

In [2]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import concurrent.futures  
import fitz
import tabula
from collections import Counter


In [3]:
wp_dir = '/content/drive/MyDrive/Research papers/Thesis /Data/whitepapers/WP_files/'

In [4]:
layout_df = pd.DataFrame(columns = ['table_count', 'image_count', 'main_font'])
layout_df['table_count'] = [None] * 1060 
layout_df['image_count'] = [None] * 1060
layout_df['main_font'] = [None] * 1060

In [5]:
names = pd.read_csv('/content/drive/MyDrive/Research papers/Thesis /Data/processed data/final_parts/TM_NER_DESC.csv')

In [6]:
layout_df = names.join(layout_df)[['name', 'table_count', 'image_count', 'main_font']]

In [63]:
def get_tables_count(i, name):
  pdf_file = wp_dir + name + '.pdf'

  dfs = tabula.read_pdf(pdf_file, pages='all')
  n_tables = len(dfs)
  
  #layout_df.loc[i, 'table_count']  = n_tables

  if i % 50 == 0:
    layout_df.to_csv(f'/content/drive/MyDrive/Research papers/Thesis /Data/processed data/final_parts/layout_{i}.csv')

  print(f"[INFO]: {i} is done!")
  return n_tables


In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, name in enumerate(layout_df.name.values.tolist()[1055:], 1055):
        executor.submit(get_tables_count, i, name)


In [14]:
layout_dir = '/content/drive/MyDrive/Research papers/Thesis /Data/processed data/final_parts/layouts/'

In [32]:
df = pd.read_csv('/content/drive/MyDrive/Research papers/Thesis /Data/processed data/final_parts/layouts/layout_100.csv')
dfs = []
for file in os.listdir(layout_dir):
  data = pd.read_csv(layout_dir + file)
  dfs.append(data)

In [37]:
df = pd.concat(dfs, axis = 0)

In [42]:
tables_df = df.dropna(subset = ['table_count']).drop_duplicates('name')

In [43]:
def get_images_and_fonts(i, name):
  pdf_file = wp_dir + name + '.pdf'
  doc = fitz.open(pdf_file)

  num_images = 0
  for page in doc:
      for image in page.get_images():
          num_images += 1

  layout_df.loc[i, 'image_count']  = num_images

  fonts = [font[0] for font in page.get_fonts() for page in doc]
  count = Counter(fonts)
  try:
    most_frequent = count.most_common(1)[0][0]
  except:
    most_frequent = None
  layout_df.loc[i, 'main_font']  = most_frequent

  print(f"[INFO]: {i} is done!")


In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, name in enumerate(layout_df.name):
        executor.submit(get_images_and_fonts, i, name)


In [49]:
tables_df = tables_df[['name', 'table_count']]

In [52]:
layout_df.drop('table_count', axis = 1, inplace = True)

In [54]:
layout_df = layout_df.merge(tables_df,  on = 'name', how = 'left')

In [65]:
layout_df.table_count.fillna(layout_df.table_count.mean(), inplace = True)

In [67]:
layout_df.main_font.fillna(9, inplace = True)


In [68]:
layout_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1060 entries, 0 to 1059
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         1060 non-null   object 
 1   image_count  1060 non-null   object 
 2   main_font    1060 non-null   int64  
 3   table_count  1060 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 41.4+ KB


In [69]:
layout_df.to_csv('/content/drive/MyDrive/Research papers/Thesis /Data/processed data/final_parts/layout.csv', index = None)